In [ ]:
import os
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/'

Mounted at /content/drive
/content/drive/My Drive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data


In [ ]:
#!ls

In [ ]:
%cd 'text'

/content/drive/My Drive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/pretrained_embeddings


# Data Preprocessing

In [ ]:




def preprocess(directory_in_str):
  directory = os.fsencode(directory_in_str)
  sentences = list()
  labels = list()  
  for file in os.listdir(directory):
      filename = os.fsdecode(file)
      with open(filename) as file:
        print(filename)
        lines = file.readlines()
        lines = [line.rstrip() for line in lines]
        for i in range(len(lines)):
          a = lines[i].split('\t')
          sent = a[0]
          label = a[1]
          sentences.append(sent)
          labels.append(label)
      file.close()

  return sentences, labels

pass

In [ ]:
%cd '/content/drive/My Drive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/pretrained_embeddings/train'
directory_in_str = '/content/drive/My Drive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/pretrained_embeddings/train'

/content/drive/My Drive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/pretrained_embeddings/train


In [ ]:
train_sentences, train_labels = preprocess(directory_in_str)

2004_C_129.txt
2006_A_136.txt
1971_S_1.txt
1987_M_123.txt
2008_A_260.txt
1996_B_72.txt
2008_I_54.txt
1994_S_246.txt
2001_A_234.txt
1987_S_26.txt
1996_T_169.txt
1987_C_108.txt
1953_S_23.txt
2008_S_1411.txt
1953_L_1.txt
1963_S_59.txt
2000_V_80.txt
2001_S_1131.txt
2008_C_166.txt
1973_S_68.txt
2006_A_36.txt
1977_P_19.txt
2000_C_151.txt
2007_S_608.txt
1980_W_3.txt
2007_S_632.txt
2008_S_549.txt
1994_M_69.txt
1995_S_317.txt
2005_S_388.txt
2008_P_8.txt
2007_B_76.txt
1954_M_25.txt
2007_U_18.txt
2009_B_16.txt
2004_I_24.txt
2007_C_121.txt
1978_M_13.txt
1976_T_9.txt
1989_A_55.txt


In [ ]:
#irectory_in_str = '/content/drive/MyDrive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/text/dev'
directory_in_str = '/content/drive/My Drive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/text/dev'
%cd '/content/drive/My Drive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/text/dev'

/content/drive/My Drive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/pretrained_embeddings/dev


In [ ]:
#%cd '/content/drive/MyDrive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/text/dev'

In [ ]:
dev_sentences, dev_labels = preprocess(directory_in_str)

2009_S_146.txt
2010_S_431.txt
2011_I_16.txt
2014_J_33.txt
2015_J_10.txt
2012_S_270.txt
2010_J_55.txt
2014_R_41.txt
2015_S_368.txt
2011_S_308.txt


## Saving Data into Json

In [ ]:

with open('train_sentences.json', 'a') as jsonfile:
      json.dump(train_sentences, jsonfile)
with open('train_labels.json', 'a') as jsonfile:
      json.dump(train_labels, jsonfile)   
with open('dev_sentences.json', 'a') as jsonfile:
      json.dump(dev_sentences, jsonfile)
with open('dev_labels.json', 'a') as jsonfile:
      json.dump(dev_labels, jsonfile)   

pass

In [ ]:
train_sentences = json.load(open('train_sentences.json'))
train_labels = json.load(open('train_labels.json'))
dev_sentences = json.load(open('dev_sentences.json'))
dev_labels = json.load(open('dev_labels.json'))

NameError: ignored

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, TrainingArguments, Trainer, DataCollator
import torch
import numpy as np
from torch.utils.data import Dataset

In [ ]:
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
class TrainingDataset(Dataset):
  
  def __init__(self):
    self.x = train_sentences
    self.y = train_labels
    self.n_samples = len(train_sentences)

  def __len__(self):
    return self.n_samples

  def __getitem__(self, index):
    input_ids = tokenizer(self.x[index])['input_ids']
    label_ids = tokenizer(self.y[index])['input_ids']
    return {"input_ids":input_ids,"labels":label_ids}

In [ ]:
class DevDataset(Dataset):
  
  def __init__(self):
    self.x = dev_sentences
    self.y = dev_labels
    self.n_samples = len(dev_sentences)

  def __len__(self):
    return self.n_samples

  def __getitem__(self, index):
    input_ids = tokenizer(self.x[index])['input_ids']
    label_ids = tokenizer(self.y[index])['input_ids']
    return {"input_ids":input_ids,"labels":label_ids}

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 8
model_name = 't5-small-paheli'
args = Seq2SeqTrainingArguments(
    f"{model_name}-seq2seq-labeling",
    evaluation_strategy = "epoch",
    learning_rate = 2e-3,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    save_total_limit = 1,
    save_strategy="no",
    num_train_epochs = 6,
    predict_with_generate = True,
    push_to_hub = False,
    load_best_model_at_end=False,
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=TrainingDataset(),
    eval_dataset=DevDataset(),
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
trainer.save_model('/content/drive/MyDrive/TUM/SS22/LDSI_LAB/Implementations/Paheli_data/text/')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
def spacy_tokenize(txt):
    doc = nlp(txt)
    tokens = list(doc)
    clean_tokens = []
    for t in tokens:
        if t.pos_ == 'PUNCT':
            pass
        elif t.pos_ == '\n' or t.pos == '\n\n':
          pass
        elif t.pos_ == 'NUM':
            clean_tokens.append(f'<NUM{len(t)}>')
        else:    
            lower_case = t.lemma_
            clean_tokens.append(lower_case)
    return clean_tokens

In [ ]:
devdataset = DevDataset()
device = 'cuda'

In [ ]:
dev_preds = list()
for i in range(len(devdataset)):
  data = devdataset.__getitem__(i)
  x = data['input_ids']
  y = data['labels']
  pred = model.generate(input_ids = torch.tensor(x).to(device).view(1,-1))
  pred_decoded = spacy_tokenize(tokenizer.decode(pred.squeeze(0)))
  pred_decoded_clean = pred_decoded[3][:-3]
  #print(pred_decoded_clean)
  dev_preds.append(pred_decoded_clean)

In [ ]:
from sklearn.metrics import f1_score
f1_weighted = f1_score(dev_labels, dev_preds, average='weighted')
f1_macro = f1_score(dev_labels, dev_preds, average='macro')
f1_micro = f1_score(dev_labels, dev_preds, average='micro')


In [ ]:
print('Weighted AVG F1: '+str(f1_weighted))
print('Macro AVG F1: '+str(f1_macro))
print('Micro AVG F1: '+str(f1_micro))

In [ ]:
### For training set:

In [ ]:
traindataset = TrainingDataset()


In [ ]:
train_preds = list()
for i in range(len(traindataset)):
  data = traindataset.__getitem__(i)
  x = data['input_ids']
  y = data['labels']
  pred = model.generate(input_ids = torch.tensor(x).to(device).view(1,-1))
  pred_decoded = spacy_tokenize(tokenizer.decode(pred.squeeze(0)))
  pred_decoded_clean = pred_decoded[3][:-3]
  #print(pred_decoded_clean)
  train_preds.append(pred_decoded_clean)

In [ ]:
from sklearn.metrics import f1_score
f1_weighted = f1_score(train_labels, train_preds, average='weighted')
f1_macro = f1_score(train_labels, train_preds, average='macro')
f1_micro = f1_score(train_labels, train_preds, average='micro')

In [ ]:
print('Weighted AVG F1: '+str(f1_weighted))
print('Macro AVG F1: '+str(f1_macro))
print('Micro AVG F1: '+str(f1_micro))